In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.
C:\Users\admprime\Miniconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admprime\Miniconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admprime\Miniconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admprime\Miniconda3\envs\k

In [2]:
import collections
import os
import cv2
import pandas as pd
import numpy as np
import random

In [3]:
NOT_ROTATE = -1
NUMBER_CLASSES = 4

def rot_corrections(label_name):
    if label_name == 'rotated_left':
        return cv2.ROTATE_90_CLOCKWISE
    elif label_name == 'upright':
        return NOT_ROTATE
    elif label_name == 'rotated_right':
        return cv2.ROTATE_90_COUNTERCLOCKWISE
    elif label_name == 'upside_down':    
        return cv2.ROTATE_180

def correct_image(image, rot_correction):
    if rot_correction == NOT_ROTATE:
        return image
    else:
        return cv2.rotate(image, rot_correction)    
    
def load_data(file_list):
    DATA_FOLDER = 'train'
    data = []
    for file in file_list:
        file_path = os.path.join(DATA_FOLDER, file)
        data.append(cv2.imread(file_path))
    data = np.array(data)
    print('Samples: ', data.shape[0],
          'x-axis width: ', data.shape[1],
          'y-axis height: ', data.shape[2],
          'colors: ', data.shape[3])
    return data
    
def preprocess(X, Y):
    """X: list of images
       Y: simple list of numbers"""
    X =X.astype('float32')
    X /= 255
    Y = np.array(Y)
    Y = Y.reshape(len(Y), 1)
    Y = keras.utils.to_categorical(Y, NUMBER_CLASSES)
    return X, Y





# LOAD DATA

In [4]:
labels = pd.read_csv('train.truth.csv', sep=',')
labels['rot_corrections'] = labels.label.apply(rot_corrections)
data_y = labels.rot_corrections.tolist()

In [5]:
data_x = load_data(labels.fn.tolist())

Samples:  48896 x-axis width:  64 y-axis height:  64 colors:  3


In [ ]:
# SHUFFLE PANDAS
#df = df.sample(frac=1).reset_index(drop=True)

# TRAIN TEST SPLIT

In [6]:
x_train = data_x[34227:]
x_test = data_x[:34227]
y_train = data_y[34227:]
y_test = data_y[:34227]

In [7]:
x_train, y_train = preprocess(x_train, y_train)
x_test, y_test = preprocess(x_test, y_test)

In [47]:
# CONFERIRI ISSO AQUI
y_test[0:7]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [48]:
data_y[:34227][0:7] 

[0, 0, -1, 2, 1, 0, 0]

In [8]:
from keras.models import model_from_json
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'rotvision_trained_model.h5'

# load json and create model
model_json_path = os.path.join(save_dir, model_name.split('.')[0] + '.json')
model_path = os.path.join(save_dir, model_name)

json_file = open(model_json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_path)

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])    

print("Loaded model from disk")

Loaded model from disk


In [9]:
# Score trained model. # era 0.78
scores = loaded_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

34227/34227 [==============================] - 351s 10ms/step
Test loss: 0.4311579054883643
Test accuracy: 0.7470125923925902


In [11]:
ypred = loaded_model.predict_classes(x_test)

In [15]:
ypred_keras = keras.utils.to_categorical(ypred, NUMBER_CLASSES)

In [13]:
y_test

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [39]:
acertos = []
erro_previsao = []
erro_teste = []
for i in range(len(y_test)):
    if str(y_test[i]) == str(ypred_keras[i]):
        acertos.append(1)
    else:
        acertos.append(0)
        erro_previsao.append(str(ypred_keras[i]))
        erro_teste.append(str(y_test[i]))


In [40]:
np.mean(acertos)

0.7470125923978146

In [41]:
collections.Counter(erro_previsao)

Counter({'[0. 0. 1. 0.]': 3392,
         '[0. 1. 0. 0.]': 502,
         '[1. 0. 0. 0.]': 4018,
         '[0. 0. 0. 1.]': 747})

In [42]:
collections.Counter(erro_teste)

Counter({'[1. 0. 0. 0.]': 3595,
         '[0. 0. 0. 1.]': 366,
         '[0. 0. 1. 0.]': 4177,
         '[0. 1. 0. 0.]': 521})

In [ ]:
#0 - prim ; -1 - ult ; 2 - terc ; 1 seg

In [50]:
cv2.ROTATE_90_COUNTERCLOCKWISE

2

In [51]:
collections.Counter(ypred)

Counter({0: 8991, 2: 7778, 1: 8523, 3: 8935})

In [ ]:
for i in range(15):
    rotated = correct_image(data[i], labels.loc[i, 'rot_corrections'])
    cv2.imwrite(str(i) + 'image.jpg', data[i])
    cv2.imwrite(str(i) + 'rotated.jpg', rotated)